# RAPTOR: Recursive Abstractive Processing for Tree-Organized Retrieval

In [2]:
# Cinderella story defined in sample.txt
with open('demo/sample.txt', 'r') as file:
    text = file.read()

print(text[:100])

The wife of a rich man fell sick, and as she felt that her end
was drawing near, she called her only


1) **Building**: RAPTOR recursively embeds, clusters, and summarizes chunks of text to construct a tree with varying levels of summarization from the bottom up. You can create a tree from the text in 'sample.txt' using `RA.add_documents(text)`.

2) **Querying**: At inference time, the RAPTOR model retrieves information from this tree, integrating data across lengthy documents at different abstraction levels. You can perform queries on the tree with `RA.answer_question`.

### Building the tree

In [3]:
from raptor import RetrievalAugmentation 

2025-01-27 22:54:15,114 - Loading faiss with AVX2 support.
2025-01-27 22:54:15,419 - Successfully loaded faiss with AVX2 support.


In [ ]:
RA = RetrievalAugmentation()

# construct the tree
RA.add_documents(text)

### Querying from the tree

```python
question = # any question
RA.answer_question(question)
```

In [ ]:
question = "How did Cinderella reach her happy ending ?"

answer = RA.answer_question(question=question)

print("Answer: ", answer)

In [ ]:
# Save the tree by calling RA.save("path/to/save")
SAVE_PATH = "demo/cinderella"
RA.save(SAVE_PATH)

In [ ]:
# load back the tree by passing it into RetrievalAugmentation

RA = RetrievalAugmentation(tree=SAVE_PATH)

answer = RA.answer_question(question=question)
print("Answer: ", answer)

## Using other Open Source Models for Summarization/QA/Embeddings

If you want to use other models such as Llama or Mistral, you can very easily define your own models and use them with RAPTOR. 

In [1]:
import torch
from raptor import BaseSummarizationModel, BaseQAModel, BaseEmbeddingModel, RetrievalAugmentationConfig, RetrievalAugmentation
from transformers import AutoTokenizer, pipeline

2025-01-28 18:01:16,680 - Loading faiss with AVX2 support.
2025-01-28 18:01:16,799 - Successfully loaded faiss with AVX2 support.


In [3]:
# if you want to use the Gemma, you will need to authenticate with HuggingFace, Skip this step, if you have the model already downloaded
from huggingface_hub import login
login()

Token has not been saved to git credential helper.
2025-01-28 13:30:31,671 - Token has not been saved to git credential helper.


In [4]:
from transformers import AutoTokenizer, pipeline
import torch

# You can define your own Summarization model by extending the base Summarization Class. 
class GEMMASummarizationModel(BaseSummarizationModel):
    def __init__(self, model_name="google/gemma-2b-it"):
        # Initialize the tokenizer and the pipeline for the GEMMA model
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.summarization_pipeline = pipeline(
            "text-generation",
            model=model_name,
            model_kwargs={"torch_dtype": torch.bfloat16},
            device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'),  # Use "cpu" if CUDA is not available
        )

    def summarize(self, context, max_tokens=150):
        # Format the prompt for summarization
        messages=[
            {"role": "user", "content": f"Write a summary of the following, including as many key details as possible: {context}:"}
        ]
        
        prompt = self.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        
        # Generate the summary using the pipeline
        outputs = self.summarization_pipeline(
            prompt,
            max_new_tokens=max_tokens,
            do_sample=True,
            temperature=0.7,
            top_k=50,
            top_p=0.95
        )
        
        # Extracting and returning the generated summary
        summary = outputs[0]["generated_text"].strip()
        return summary


In [5]:
class GEMMAQAModel(BaseQAModel):
    def __init__(self, model_name= "google/gemma-2b-it"):
        # Initialize the tokenizer and the pipeline for the model
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.qa_pipeline = pipeline(
            "text-generation",
            model=model_name,
            model_kwargs={"torch_dtype": torch.bfloat16},
            device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
        )

    def answer_question(self, context, question):
        # Apply the chat template for the context and question
        messages=[
              {"role": "user", "content": f"Given Context: {context} Give the best full answer amongst the option to question {question}"}
        ]
        prompt = self.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        
        # Generate the answer using the pipeline
        outputs = self.qa_pipeline(
            prompt,
            max_new_tokens=256,
            do_sample=True,
            temperature=0.7,
            top_k=50,
            top_p=0.95
        )
        
        # Extracting and returning the generated answer
        answer = outputs[0]["generated_text"][len(prompt):]
        return answer

In [2]:
from sentence_transformers import SentenceTransformer
class SBertEmbeddingModel(BaseEmbeddingModel):
    def __init__(self, model_name="BAAI/bge-small-en-v1.5"):
        self.model = SentenceTransformer(model_name)

    def create_embedding(self, text):
        return self.model.encode(text)


In [8]:
RAC = RetrievalAugmentationConfig(summarization_model=GEMMASummarizationModel(), qa_model=GEMMAQAModel(), embedding_model=SBertEmbeddingModel())

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cpu


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cpu
2025-01-28 11:25:09,865 - Use pytorch device_name: cpu
2025-01-28 11:25:09,868 - Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5


In [9]:
RA = RetrievalAugmentation(config=RAC)

2025-01-28 11:25:25,518 - Successfully initialized TreeBuilder with Config 
        TreeBuilderConfig:
            Tokenizer: <Encoding 'cl100k_base'>
            Max Tokens: 100
            Num Layers: 5
            Threshold: 0.5
            Top K: 5
            Selection Mode: top_k
            Summarization Length: 100
            Summarization Model: <__main__.GEMMASummarizationModel object at 0x000001AB0D0CF790>
            Embedding Models: {'EMB': <__main__.SBertEmbeddingModel object at 0x000001AB18E50A60>}
            Cluster Embedding Model: EMB
        
        Reduction Dimension: 10
        Clustering Algorithm: RAPTOR_Clustering
        Clustering Parameters: {}
        
2025-01-28 11:25:25,520 - Successfully initialized ClusterTreeBuilder with Config 
        TreeBuilderConfig:
            Tokenizer: <Encoding 'cl100k_base'>
            Max Tokens: 100
            Num Layers: 5
            Threshold: 0.5
            Top K: 5
            Selection Mode: top_k
            

In [18]:
with open('demo/sample.txt', 'r') as file:
    text = file.read()
    
RA.add_documents(text)

2025-01-27 18:47:30,953 - Creating Leaf Nodes


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-01-27 18:47:44,908 - Created 35 Leaf Embeddings
2025-01-27 18:47:44,910 - Building All Nodes
2025-01-27 18:47:44,919 - Using Cluster TreeBuilder
2025-01-27 18:47:44,921 - Constructing Layer 0
i:\AI_Project\raptor\.venv\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
i:\AI_Project\raptor\.venv\lib\site-packages\threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)
2025-01-27 18:48:12,514 - Summarization Length: 100


In [ ]:
question = "How did Cinderella reach her happy ending?"

answer = RA.answer_question(question=question)

print("Answer: ", answer)

In [3]:
from raptor import OllamaQAModelV2, OllamaSummarizationModelV2
qa_llm = 'llama3.2:latest'
qa_model = OllamaQAModelV2(model_name= qa_llm)
summarization_llm = 'llama3.2:latest'
summarization_model = OllamaSummarizationModelV2(model_name= summarization_llm)

RAC = RetrievalAugmentationConfig(qa_model=qa_model, summarization_model=summarization_model, embedding_model=SBertEmbeddingModel())
RA = RetrievalAugmentation(config=RAC, tree="demo/football360")

2025-01-28 18:01:31,000 - Use pytorch device_name: cpu
2025-01-28 18:01:31,003 - Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5
2025-01-28 18:01:37,385 - Successfully initialized TreeBuilder with Config 
        TreeBuilderConfig:
            Tokenizer: <Encoding 'cl100k_base'>
            Max Tokens: 100
            Num Layers: 5
            Threshold: 0.5
            Top K: 5
            Selection Mode: top_k
            Summarization Length: 100
            Summarization Model: <raptor.SummarizationModels.OllamaSummarizationModelV2 object at 0x000001E7B8173430>
            Embedding Models: {'EMB': <__main__.SBertEmbeddingModel object at 0x000001E7B8173370>}
            Cluster Embedding Model: EMB
        
        Reduction Dimension: 10
        Clustering Algorithm: RAPTOR_Clustering
        Clustering Parameters: {}
        
2025-01-28 18:01:37,387 - Successfully initialized ClusterTreeBuilder with Config 
        TreeBuilderConfig:
            Tokenizer: <Encoding '

In [5]:
f = open("demo/data4.txt", "r", encoding="utf-8")
text = ""
for x in f:
  text += x

In [6]:
RA.add_documents(text)

2025-01-28 17:00:41,397 - Creating Leaf Nodes


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

2025-01-28 17:45:46,143 - Node Texts Length: 1567, Summarized Text Length: 1429


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-01-28 17:46:28,132 - Node Texts Length: 409, Summarized Text Length: 175


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-01-28 17:46:28,394 - Constructing Layer 2
2025-01-28 17:46:28,396 - Stopping Layer construction: Cannot Create More Layers. Total Layers in tree: 2
2025-01-28 17:46:28,430 - Successfully initialized TreeRetriever with Config 
        TreeRetrieverConfig:
            Tokenizer: <Encoding 'cl100k_base'>
            Threshold: 0.5
            Top K: 5
            Selection Mode: top_k
            Context Embedding Model: EMB
            Embedding Model: <__main__.SBertEmbeddingModel object at 0x000001B3EAABC1F0>
            Num Layers: None
            Start Layer: None
        


In [4]:
question = "خریدهای زمستانی استقلال چه کسانی بودند؟"
answer = RA.answer_question(question=question, return_layer_information=True)
print("Answer: ", answer)

2025-01-28 18:01:57,129 - Using collapsed_tree


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Answer:  ('هشتاد و چهارمین خرید زمستانی استقلال سهرابیان  بود.', [{'node_index': 2, 'layer_number': 0}, {'node_index': 92, 'layer_number': 0}, {'node_index': 57, 'layer_number': 0}, {'node_index': 73, 'layer_number': 0}, {'node_index': 28, 'layer_number': 0}, {'node_index': 6, 'layer_number': 0}, {'node_index': 15, 'layer_number': 0}, {'node_index': 25, 'layer_number': 0}, {'node_index': 67, 'layer_number': 0}, {'node_index': 19, 'layer_number': 0}])


In [7]:
SAVE_PATH = "demo/football360"
RA.save(SAVE_PATH)

2025-01-28 17:47:21,635 - Tree successfully saved to demo/football360


In [5]:
## Now see the tree build in action
RA.visualize_tree()